## Fault-Tolerance

One of the most important aspects of the MapReduce distributed system is fault-tolerance.

If <ins>at all times at least one</ins> *master* service is alive, we guarantee fault tolerance for the following scenarios:
1. *Worker* failures at any time.
2. *Master* failures at any time.

Our system not only guarantees fault-tolerance, but also that the computations in the event of *worker*, *master* failures will continue exactly where they left off as if nothing bad has happened. No recomputation will take place if not necessary. 

This notebook aims to simulate an extreme crisis situation in our MapReduce system: a perfect storm. Imagine a scenario where all the workers, in the midst of executing a MapReduce job, abruptly fail. As if this isn't catastrophic enough, the master responsible for coordinating that very job also shuts down unexpectedly, leaving a lone master as the sole survivor in our distributed system.

Note: Addressing *worker* failures alone was comparatively simple. Likewise, overcoming *master* failures presented its unique challenges, that forced us to revise certain parts of our system. However, managing the simultaneous failures of both *workers* and *masters* - especially within the same job - has been an entirely different level of complexity. This scenario necessitates an extremely robust and meticulous system implementation. There are numerous intricate edge-cases that open the door for potential race condition issues. The system must not only handle failures independently but also efficiently coordinate and synchronize their concurrent occurrence.

### Authenticate

Use the `Auth` in-between interface for fast authentication.

In [57]:
from mapreduce.authentication.auth import Auth

In [58]:
auth = Auth(username='admin', password='admin')

In [59]:
auth.is_authenticated()

True

### Initialize the docker-compose network 

In [60]:
from mapreduce.cluster.local_cluster import LocalCluster

In [61]:
cluster = LocalCluster(
    auth=auth,
    n_workers=4,
    n_masters=2,
    initialize=True,
    verbose=False
)

As in the previous notebook, let's assume that our objective is to count how many times each character appears in the a list of words.

In [62]:
def map_func(data):
    result = []
    for string in data:
        for char in string:
            result.append((char, 1))
    return result

def reduce_func(data):
    return sum(data)

Create some reasonable amount of data.

In [63]:
import random
import string

def generate_random_string(str_len):
    letters = string.ascii_lowercase
    return ''.join(random.choice(letters) for _ in range(str_len))

def generate_random_string_list(n, str_len):
    return [generate_random_string(str_len) for _ in range(n)]

data = generate_random_string_list(n=500_000, str_len=20)

In [64]:
data[:10]

['wqiweuzzpwcvtvanlsve',
 'dqltqatyejlsqcqwrnow',
 'nnggnimkcdeqddcqurpo',
 'rwelramzkrbvppoovnig',
 'yzbrnkaqpnlvugjxnhho',
 'tgsjtubksqzefttgwnhe',
 'eufooahdkkpdhnzksiqv',
 'zygbtgxuqisvgarrawqp',
 'wxyhkgragleecinlfbco',
 'dkuogphgkpijqbtogany']

### Kill workers amidst computation

This part (the next one aswell) will not be entirely sequential. We have configured `docker-compose` so that the *workers* and *masters* use the *container-id* as *hostname*. Hence, grab the 4 *worker* container-ids from bellow and prepare the `docker kill` statement.

In [65]:
cluster.local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master 455251675c09 :  MasterInfo(state='nothing')
Master 9b4a284e74be :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker c3aa3f9c75ce :  WorkerInfo(state='idle')
Worker 113b86cfa5d5 :  WorkerInfo(state='idle')
Worker 8d95f0c5afad :  WorkerInfo(state='idle')
Worker 5033b9dc545b :  WorkerInfo(state='idle')

----------------- Zoo Map Tasks -----------------

----------------- Zoo Shuffle Tasks -----------------

----------------- Zoo Reduce Tasks -----------------

----------------- Zoo Jobs ---------------------


------------- Dead Worker Tasks ---------------------


In [66]:
cluster.local_monitoring.print_hdfs('jobs')

Note, if `requested_n_workers` is not provided it defaults to `None` which is translated to the underlying computation as "use as many workers as possible".

By now, you should have replaced the `docker kill` command with the appropriate ids from above (load the gun). We are ready to submit the job and kill the 4 workers on the spot.

In [67]:
import time

future = cluster.mapreduce(
    data=data, 
    map_func=map_func, 
    reduce_func=reduce_func
)

# ------- Kill workers on specific point in the MapReduce execution -----

# Get the Zookeeper client
zk_client = cluster.get_zk_client()

# Put `/reduce_tasks` if you want to kill on reduce, `/shuffle_tasks` on shuffle
kill_workers_amidst = '/map_tasks'

def tasks_received(file_tasks):
    return all(zk_client.get(f'{file_tasks}/{file}').received for file in zk_client.zk.get_children(file_tasks))
        
while not zk_client.zk.get_children(kill_workers_amidst) or not tasks_received(kill_workers_amidst):
    time.sleep(0.05)

# ------- Kill workers now -----

# This is shell (REPLACE from above)
!docker kill c3aa3f9c75ce 113b86cfa5d5 8d95f0c5afad 5033b9dc545b

c3aa3f9c75ce
113b86cfa5d5
8d95f0c5afad
5033b9dc545b


In [68]:
future

<Future at 0x7f523a069110 state=running>

We have now killed the 4 workers. Let's take a look at the state of the computation.

In [70]:
cluster.local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master 455251675c09 :  MasterInfo(state='nothing')
Master 9b4a284e74be :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------

----------------- Zoo Map Tasks -----------------
Task 0_0 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_1 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_2 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_3 :  Task(state='in-progress', worker_hostname='None', received=True)

----------------- Zoo Shuffle Tasks -----------------

----------------- Zoo Reduce Tasks -----------------

----------------- Zoo Jobs ---------------------
Job 0 :  Job(state='in-progress', requested_n_workers=None, master_hostname='455251675c09')


------------- Dead Worker Tasks ---------------------
Dead Worker Task 0_0 :  DeadTask(state='in-progress', master_hostname='455251675c09', task_type='map')
Dead Worker Task 0_1 :  DeadTas

### Kill the master handling the job

Observe that our cluster is currently bereft of workers, thus bringing the computation to a standstill. But we're not stopping there. Let's intensify this scenario further. We'll seek out the master overseeing the job and kill him. In doing so, we'll bring our system to its most fundamental state required to test our fault-tolerance guarantees - that is, maintaining at least one surviving master in the midst of chaos.

In [71]:
# REPLACE from above
!docker kill 455251675c09

455251675c09


In [75]:
cluster.local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master 9b4a284e74be :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------

----------------- Zoo Map Tasks -----------------
Task 0_0 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_1 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_2 :  Task(state='in-progress', worker_hostname='None', received=True)
Task 0_3 :  Task(state='in-progress', worker_hostname='None', received=True)

----------------- Zoo Shuffle Tasks -----------------

----------------- Zoo Reduce Tasks -----------------

----------------- Zoo Jobs ---------------------
Job 0 :  Job(state='in-progress', requested_n_workers=None, master_hostname='9b4a284e74be')


------------- Dead Worker Tasks ---------------------
Dead Worker Task 0_0 :  DeadTask(state='in-progress', master_hostname='9b4a284e74be', task_type='map')
Dead Worker Task 0_1 :  DeadTask(state='in-progress', master_hostname='9b4a284e74b

Notice that all the *responsibilities* of the dead master have transfered to the last surviving master (dead *worker* tasks and the job). For now, our system is in a bit of a freeze. 

### Scale the system

Let's jumpstart the system by scaling it and get the computation back on track. We will add 10 workers to the system so the computation terminates quickly (remember that we passed `requested_n_workers` as `None` - use as many workers as possible).

Note: We do not have to pass `n_masters=2`.

In [76]:
cluster.scale(n_masters=2, n_workers=10)

In [79]:
cluster.local_monitoring.print_zoo()


----------------- Zoo Masters -----------------
Master 455251675c09 :  MasterInfo(state='nothing')
Master 9b4a284e74be :  MasterInfo(state='nothing')

----------------- Zoo Workers -----------------
Worker c3aa3f9c75ce :  WorkerInfo(state='idle')
Worker 21f2333b30e5 :  WorkerInfo(state='idle')
Worker 3f4f12cc3a83 :  WorkerInfo(state='idle')
Worker 113b86cfa5d5 :  WorkerInfo(state='idle')
Worker ee9bcfbcdb52 :  WorkerInfo(state='idle')
Worker a8ae1bcf033b :  WorkerInfo(state='idle')
Worker c6e634cb5e76 :  WorkerInfo(state='idle')
Worker 4e486cfab832 :  WorkerInfo(state='idle')
Worker 8d95f0c5afad :  WorkerInfo(state='idle')
Worker 5033b9dc545b :  WorkerInfo(state='idle')

----------------- Zoo Map Tasks -----------------
Task 0_0 :  Task(state='completed', worker_hostname='113b86cfa5d5', received=True)
Task 0_1 :  Task(state='completed', worker_hostname='21f2333b30e5', received=True)
Task 0_2 :  Task(state='completed', worker_hostname='3f4f12cc3a83', received=True)
Task 0_3 :  Task(sta

In [80]:
future.result()

[('a', 384908),
 ('b', 384710),
 ('c', 384655),
 ('m', 385397),
 ('n', 384379),
 ('o', 384750),
 ('p', 384546),
 ('q', 383329),
 ('r', 385062),
 ('s', 384774),
 ('t', 384563),
 ('u', 384552),
 ('v', 384386),
 ('w', 384494),
 ('x', 384269),
 ('y', 384779),
 ('z', 383976),
 ('d', 384408),
 ('e', 384083),
 ('f', 384732),
 ('g', 384015),
 ('h', 384564),
 ('i', 385879),
 ('j', 384868),
 ('k', 384441),
 ('l', 385481)]

### Shutdown

In [81]:
cluster.shutdown_cluster(cleanup=True)